In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def rm_outlier(df):
    y_p75 = np.percentile(df[rate], 75)
    y_p25 = np.percentile(df[rate], 25)
    upper_bound = (y_p75-y_p25)*1.5+y_p75
    return df[df[rate] <= upper_bound]

In [3]:
def train_test_split_df(df, test_size=0.2):
    test_df = df.sample(frac = test_size)
    train_df = df.drop(test_df.index)
    return train_df, test_df

In [4]:
train_data = pd.read_csv('data/rain_train.csv')#.drop(['tumbol_name','amphur_name','province_name'], axis=1)
rate = 'poverty_rate_living'
pov_rate = [c for c in train_data.columns if 'poverty_rate' in c and c != rate]
train_data = train_data.drop(pov_rate, axis=1).dropna()
# train_data['LAT'] = np.cos(train_data['LAT'])
# train_data['LONG'] = np.cos(train_data['LONG'])
train_data

,tumbol_ID,sum_rain,poverty_rate_living
0,110113,2414.99,0.290314
1,110115,2414.99,0.070353
2,110117,2414.99,0.019508
7,110201,2835.25,0.824092
8,110202,2835.25,0.077022
...,...,...,...
4546,961105,2863.12,1.236233
4547,961201,4181.46,12.716118
4548,961202,3668.46,0.044823
4549,961302,2863.12,4.445484


In [5]:
train_df, test_df = train_test_split_df(train_data)
print(train_df.shape, test_df.shape)
train_df = rm_outlier(train_df)
print(train_df.shape, test_df.shape)

(3362, 3) (841, 3)
(2859, 3) (841, 3)


In [6]:
X_train = train_df.iloc[:, 1:-1].values
y_train = train_df.iloc[:, -1].values
print(X_train.shape, y_train.shape)

X_test = test_df.iloc[:, 1:-1].values
y_test = test_df.iloc[:, -1].values
print(X_test.shape, y_test.shape)

(2859, 1) (2859,)
(841, 1) (841,)


In [7]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=None)

# X_train_df
# print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import HuberRegressor
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor, ExtraTreesClassifier, ExtraTreesRegressor
# from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklego.meta import ZeroInflatedRegressor

X_scaler = StandardScaler()
y_scaler = StandardScaler()

X_train = X_scaler.fit_transform(X_train)
X_test = X_scaler.transform(X_test)

y_train = y_scaler.fit_transform(y_train.reshape(-1, 1)).reshape(-1)

In [9]:
# from sklearn.decomposition import PCA

# pca = PCA(n_components=100)
# X_train = pca.fit_transform(X_train)
# X_test = pca.transform(X_test)

In [10]:
zir = ZeroInflatedRegressor(
    classifier=ExtraTreesClassifier(),
    regressor=HuberRegressor(max_iter=1000)
)

zir.fit(X_train, y_train)

ZeroInflatedRegressor(classifier=ExtraTreesClassifier(),
                      regressor=HuberRegressor(max_iter=1000))

In [11]:
y_pred = zir.predict(X_test)
y_pred = y_scaler.inverse_transform(y_pred.reshape(-1, 1))

In [12]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(y_test, y_pred)
mae

2.166838164660649

In [13]:
zero = mean_absolute_error(y_test, [0]*len(y_test))
zero

2.2122347422211655

In [14]:
(zero-mae)/zero

0.020520687381908067

In [ ]:
test_data = pd.read_csv('data/dist_test.csv')#.drop(['tumbol_name','amphur_name','province_name'], axis=1)
test_data

In [ ]:
test_rm_nan = test_data.dropna()
test_rm_nan

In [ ]:
X_submit = test_rm_nan.iloc[:, 1:].values
X_submit = X_scaler.transform(X_submit)
# X_submit = pca.transform(X_submit)

In [ ]:
y_submit = zir.predict(X_submit)
y_submit = y_scaler.inverse_transform(y_submit.reshape(-1, 1))

In [ ]:
test_rm_nan[rate] = y_submit
submit_df = test_data.merge(test_rm_nan, on= 'tumbol_ID', how= 'left')[['tumbol_ID', 'poverty_rate_living']].fillna(0)
submit_df

In [ ]:
submit_df.to_csv('submit_living_3.csv', index=False)